# Stock Market Analysis
## Marshall Ferguson 9/2021

In [1]:
# Imports

import os
import csv
import numpy as np
import pandas as pd

In [2]:
# Read in data

df = pd.read_csv('data/sp500/AAPL.csv')

df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.100600
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.095352
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.088353
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.090540
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.093165


As an aside, the dates in the Date Series are in DD-MM-YYY format

This data is set up differently than I would like. It is organized into folders based on whether the company is part of NASDAQ, NYSE, or S&P500. Within those folders are seperate CSV files for each company. I would like to combine these seperate files into one master dataset. My thought process is that I can turn each CSV into a dict, and then turn it into one giant df from combining each dict.

In [3]:
# Trial run converting AAPL csv to a list of dicts

AAPL = []

with open('data/sp500/AAPL.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['Ticker'] = 'AAPL'
        AAPL.append(row)

AAPL[0]

{'Date': '12-12-1980',
 'Low': '0.1283479928970337',
 'Open': '0.1283479928970337',
 'Volume': '469033600',
 'High': '0.1289059966802597',
 'Close': '0.1283479928970337',
 'Adjusted Close': '0.10060006380081177',
 'Ticker': 'AAPL'}

In [4]:
# Convert all rows in all files in sp500 directory to dicts
# and append said dicts to one list, after adding Ticker to dict

sp500 = []

for file in os.listdir('data/sp500'):
    with open(f'data/sp500/{file}', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            file_name = os.path.splitext(file)
            row['Ticker'] = file_name[0]
            sp500.append(row)

In [5]:
sp500[0]

{'Date': '18-11-1999',
 'Low': '28.612302780151367',
 'Open': '32.54649353027344',
 'Volume': '62546380',
 'High': '35.765380859375',
 'Close': '31.473533630371094',
 'Adjusted Close': '27.135984420776367',
 'Ticker': 'A'}

In [6]:
sp500[-1]

{'Date': '30-08-2021',
 'Low': '204.0050048828125',
 'Open': '203.82000732421875',
 'Volume': '288506.0',
 'High': '206.97999572753906',
 'Close': '205.75',
 'Adjusted Close': '205.75',
 'Ticker': 'ZTS'}

In [7]:
# Convert list of sp500 dicts to df

sp500_df = pd.DataFrame(sp500)

In [8]:
sp500_df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
0,18-11-1999,28.612302780151367,32.54649353027344,62546380,35.765380859375,31.473533630371094,27.135984420776367,A
1,19-11-1999,28.47818374633789,30.713518142700195,15234146,30.75822639465332,28.880544662475586,24.900341033935547,A
2,22-11-1999,28.65700912475586,29.551143646240234,6577870,31.473533630371094,31.473533630371094,27.135984420776367,A
3,23-11-1999,28.612302780151367,30.400571823120117,5975611,31.205293655395508,28.612302780151367,24.669065475463867,A
4,24-11-1999,28.612302780151367,28.701717376708984,4843231,29.998212814331055,29.372318267822266,25.324344635009766,A


In [9]:
sp500_df.tail()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
3221722,24-08-2021,204.49000549316406,206.08999633789062,97870000.0,206.9600067138672,204.99000549316406,204.99000549316406,ZTS
3221723,25-08-2021,202.97999572753906,204.38999938964844,78070000.0,205.0800018310547,204.42999267578125,204.42999267578125,ZTS
3221724,26-08-2021,203.36000061035156,204.8000030517578,95760000.0,205.49000549316406,204.02000427246094,204.02000427246094,ZTS
3221725,27-08-2021,203.22999572753906,204.67999267578125,85530000.0,206.19000244140625,203.82000732421875,203.82000732421875,ZTS
3221726,30-08-2021,204.0050048828125,203.82000732421875,288506.0,206.97999572753906,205.75,205.75,ZTS


Now that we have one dataset for all the S&P500 data, let's do some data cleaning...

In [10]:
# What types of data are in the dataframe?

sp500_df.dtypes

Date              object
Low               object
Open              object
Volume            object
High              object
Close             object
Adjusted Close    object
Ticker            object
dtype: object

We need to convert the Series' that represent stock prices into numeric values

In [11]:
numb_convert = ['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close']

for series in numb_convert:
    sp500_df[series] = pd.to_numeric(sp500_df[series])

In [12]:
sp500_df.dtypes

Date               object
Low               float64
Open              float64
Volume            float64
High              float64
Close             float64
Adjusted Close    float64
Ticker             object
dtype: object

Now that the numeric data is treated as numerics by Python, let's do some analysis...

### Which company has the highest ATH(All Time High)?

In [39]:
# Find highest value in High Series
higest_ath = sp500_df['High'].max()
# Find index of highest value in High Series
highest_ath_index = sp500_df['High'].idxmax()
# Find stock ticker for highest value
ticker_highest_ath = sp500_df.iloc[highest_ath_index]['Ticker']
# Find date at which the stock was at ATH
date_highest_ath = sp500_df.iloc[highest_ath_index]['Date']

print(f'The company with the highest ATH in the dataset is {ticker_highest_ath} with a price of {higest_ath} on {date_highest_ath}.')

The company with the highest ATH in the dataset is BRK-A with a price of 445000.0 on 07-05-2021.


### Which company has the lowest ATL(All Time Low)?

In [42]:
# Find lowest value in Low Series
lowest_atl = sp500_df['Low'].min()
# Find index of lowest value in Low Series
lowest_atl_index = sp500_df['Low'].idxmin()
# Find stock ticker for lowest value
ticker_lowest_atl = sp500_df.iloc[lowest_atl_index]['Ticker']
# Find date at which the stock was at ATL
date_lowest_atl = sp500_df.iloc[lowest_atl_index]['Date']

print(f'The company with the lowest ATL in the dataset is {ticker_lowest_atl} with a price of {lowest_atl} on {date_lowest_atl}.')

The company with the lowest ATL in the dataset is NEOG with a price of 0.0 on 05-08-1993.
